## Table Extraction

## Setup and Imports

In [25]:
# Table Extraction for Financial Documents
# Comparing Camelot (lattice/stream) vs pdfplumber table detection

import os
import glob
import json
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Tuple, Optional
import matplotlib.pyplot as plt
import seaborn as sns

# Table extraction libraries
import camelot
import pdfplumber
from pdfplumber import PDF

# For visualization and analysis
import warnings
warnings.filterwarnings('ignore')

### Configuration and Path Setup

In [26]:
# Define paths
raw_pdf_dir = "../data/raw/MSFT/10-K/PDFs"
table_output_dir = "../data/tables/MSFT"

# Create output directory structure
os.makedirs(table_output_dir, exist_ok=True)

# Table extraction settings
CAMELOT_SETTINGS = {
    'lattice': {
        'line_scale': 40,
        'copy_text': ['v'],
        'split_text': True,
        'flag_size': True,
        'strip_text': '\n'
    },
    'stream': {
        'edge_tol': 500,
        'row_tol': 10,
        'column_tol': 10,
        'copy_text': ['v'],
        'split_text': True,
        'flag_size': True,
        'strip_text': '\n'
    }
}

PDFPLUMBER_SETTINGS = {
    'vertical_strategy': 'lines',
    'horizontal_strategy': 'lines',
    'snap_tolerance': 3,
    'join_tolerance': 3,
    'edge_min_length': 3,
    'min_words_vertical': 3,
    'min_words_horizontal': 1
}

### Data Classes for Table Analysis

In [27]:
from dataclasses import dataclass, asdict
from enum import Enum

class TableMethod(Enum):
    CAMELOT_LATTICE = "camelot_lattice"
    CAMELOT_STREAM = "camelot_stream"
    PDFPLUMBER = "pdfplumber"

@dataclass
class TableExtractionResult:
    """Data class for table extraction results"""
    method: TableMethod
    page_number: int
    table_index: int
    confidence: Optional[float]
    accuracy: Optional[float]
    rows: int
    columns: int
    raw_data: pd.DataFrame
    extraction_time: float
    success: bool
    error_message: Optional[str] = None
    
    def to_dict(self):
        return {
            'method': self.method.value,
            'page_number': self.page_number,
            'table_index': self.table_index,
            'confidence': self.confidence,
            'accuracy': self.accuracy,
            'rows': self.rows,
            'columns': self.columns,
            'extraction_time': self.extraction_time,
            'success': self.success,
            'error_message': self.error_message
        }

@dataclass
class TableComparison:
    """Data class for comparing table extraction methods"""
    page_number: int
    table_index: int
    methods_compared: List[TableMethod]
    best_method: Optional[TableMethod]
    quality_scores: Dict[str, float]
    row_column_preservation: Dict[str, Dict[str, int]]
    recommendations: List[str]
    
    def to_dict(self):
        return asdict(self)

### Data Classes for Table Analysis

In [28]:
from dataclasses import dataclass, asdict
from enum import Enum

class TableMethod(Enum):
    CAMELOT_LATTICE = "camelot_lattice"
    CAMELOT_STREAM = "camelot_stream"
    PDFPLUMBER = "pdfplumber"

@dataclass
class TableExtractionResult:
    """Data class for table extraction results"""
    method: TableMethod
    page_number: int
    table_index: int
    confidence: Optional[float]
    accuracy: Optional[float]
    rows: int
    columns: int
    raw_data: pd.DataFrame
    extraction_time: float
    success: bool
    error_message: Optional[str] = None
    
    def to_dict(self):
        return {
            'method': self.method.value,
            'page_number': self.page_number,
            'table_index': self.table_index,
            'confidence': self.confidence,
            'accuracy': self.accuracy,
            'rows': self.rows,
            'columns': self.columns,
            'extraction_time': self.extraction_time,
            'success': self.success,
            'error_message': self.error_message
        }

@dataclass
class TableComparison:
    """Data class for comparing table extraction methods"""
    page_number: int
    table_index: int
    methods_compared: List[TableMethod]
    best_method: Optional[TableMethod]
    quality_scores: Dict[str, float]
    row_column_preservation: Dict[str, Dict[str, int]]
    recommendations: List[str]
    
    def to_dict(self):
        return {
            'page_number': self.page_number,
            'table_index': self.table_index,
            'methods_compared': [method.value for method in self.methods_compared],  # Convert enum to string
            'best_method': self.best_method.value if self.best_method else None,  # Convert enum to string
            'quality_scores': self.quality_scores,
            'row_column_preservation': self.row_column_preservation,
            'recommendations': self.recommendations
        }

### Camelot Table Extraction Functions
 


In [29]:
def extract_tables_camelot_lattice(pdf_path: str, page_num: int = None) -> List[TableExtractionResult]:
    """Extract tables using Camelot lattice mode"""
    results = []
    
    try:
        start_time = datetime.now()
        
        # Extract tables with lattice mode
        tables = camelot.read_pdf(
            pdf_path, 
            pages=str(page_num) if page_num else 'all',
            flavor='lattice',
            **CAMELOT_SETTINGS['lattice']
        )
        
        extraction_time = (datetime.now() - start_time).total_seconds()
        
        for i, table in enumerate(tables):
            result = TableExtractionResult(
                method=TableMethod.CAMELOT_LATTICE,
                page_number=table.page,
                table_index=i,
                confidence=table.accuracy,
                accuracy=table.accuracy,
                rows=len(table.df),
                columns=len(table.df.columns),
                raw_data=table.df,
                extraction_time=extraction_time,
                success=True
            )
            results.append(result)
            
    except Exception as e:
        result = TableExtractionResult(
            method=TableMethod.CAMELOT_LATTICE,
            page_number=page_num or 0,
            table_index=0,
            confidence=None,
            accuracy=None,
            rows=0,
            columns=0,
            raw_data=pd.DataFrame(),
            extraction_time=0,
            success=False,
            error_message=str(e)
        )
        results.append(result)
    
    return results

def extract_tables_camelot_stream(pdf_path: str, page_num: int = None) -> List[TableExtractionResult]:
    """Extract tables using Camelot stream mode"""
    results = []
    
    try:
        start_time = datetime.now()
        
        # Extract tables with stream mode
        tables = camelot.read_pdf(
            pdf_path, 
            pages=str(page_num) if page_num else 'all',
            flavor='stream',
            **CAMELOT_SETTINGS['stream']
        )
        
        extraction_time = (datetime.now() - start_time).total_seconds()
        
        for i, table in enumerate(tables):
            result = TableExtractionResult(
                method=TableMethod.CAMELOT_STREAM,
                page_number=table.page,
                table_index=i,
                confidence=table.accuracy,
                accuracy=table.accuracy,
                rows=len(table.df),
                columns=len(table.df.columns),
                raw_data=table.df,
                extraction_time=extraction_time,
                success=True
            )
            results.append(result)
            
    except Exception as e:
        result = TableExtractionResult(
            method=TableMethod.CAMELOT_STREAM,
            page_number=page_num or 0,
            table_index=0,
            confidence=None,
            accuracy=None,
            rows=0,
            columns=0,
            raw_data=pd.DataFrame(),
            extraction_time=0,
            success=False,
            error_message=str(e)
        )
        results.append(result)
    
    return results

### PDFplumber Table Extraction Function

In [30]:
def extract_tables_pdfplumber(pdf_path: str, page_num: int = None) -> List[TableExtractionResult]:
    """Extract tables using pdfplumber"""
    results = []
    
    try:
        start_time = datetime.now()
        
        with pdfplumber.open(pdf_path) as pdf:
            pages_to_process = [page_num] if page_num else range(len(pdf.pages))
            
            for page_idx in pages_to_process:
                page = pdf.pages[page_idx]
                
                # Extract tables from the page
                tables = page.extract_tables(**PDFPLUMBER_SETTINGS)
                
                extraction_time = (datetime.now() - start_time).total_seconds()
                
                for i, table in enumerate(tables):
                    if table:  # Check if table is not empty
                        # Convert to DataFrame
                        df = pd.DataFrame(table[1:], columns=table[0] if table[0] else None)
                        
                        result = TableExtractionResult(
                            method=TableMethod.PDFPLUMBER,
                            page_number=page_idx + 1,
                            table_index=i,
                            confidence=None,  # pdfplumber doesn't provide confidence
                            accuracy=None,
                            rows=len(df),
                            columns=len(df.columns),
                            raw_data=df,
                            extraction_time=extraction_time,
                            success=True
                        )
                        results.append(result)
                        
    except Exception as e:
        result = TableExtractionResult(
            method=TableMethod.PDFPLUMBER,
            page_number=page_num or 0,
            table_index=0,
            confidence=None,
            accuracy=None,
            rows=0,
            columns=0,
            raw_data=pd.DataFrame(),
            extraction_time=0,
            success=False,
            error_message=str(e)
        )
        results.append(result)
    
    return results

### Table Quality Assessment Functions

In [31]:
def assess_table_quality(df: pd.DataFrame, method: TableMethod) -> Dict[str, float]:
    """Assess the quality of an extracted table"""
    if df.empty:
        return {'completeness': 0, 'structure_preservation': 0, 'data_quality': 0}
    
    # Completeness: percentage of non-empty cells
    total_cells = df.size
    non_empty_cells = df.notna().sum().sum()
    completeness = (non_empty_cells / total_cells) * 100 if total_cells > 0 else 0
    
    # Structure preservation: consistency in row/column structure
    row_lengths = df.notna().sum(axis=1)
    structure_preservation = 100 - (row_lengths.std() / row_lengths.mean() * 100) if row_lengths.mean() > 0 else 0
    structure_preservation = max(0, structure_preservation)
    
    # Data quality: based on method-specific heuristics
    data_quality = 0
    if method == TableMethod.CAMELOT_LATTICE:
        # Lattice mode is good for bordered tables
        data_quality = min(100, completeness + 10)
    elif method == TableMethod.CAMELOT_STREAM:
        # Stream mode is good for borderless tables
        data_quality = min(100, completeness + 5)
    else:  # PDFplumber
        # PDFplumber is good for complex layouts
        data_quality = min(100, completeness + 15)
    
    return {
        'completeness': completeness,
        'structure_preservation': structure_preservation,
        'data_quality': data_quality,
        'overall_score': (completeness + structure_preservation + data_quality) / 3
    }

def detect_table_type(df: pd.DataFrame) -> str:
    """Detect the type of financial table"""
    if df.empty:
        return 'empty'
    
    # Look for common financial statement indicators
    text_content = ' '.join(df.astype(str).values.flatten()).lower()
    
    if any(keyword in text_content for keyword in ['balance sheet', 'assets', 'liabilities', 'equity']):
        return 'balance_sheet'
    elif any(keyword in text_content for keyword in ['income statement', 'revenue', 'expenses', 'net income']):
        return 'income_statement'
    elif any(keyword in text_content for keyword in ['cash flow', 'operating activities', 'investing activities']):
        return 'cash_flow_statement'
    elif any(keyword in text_content for keyword in ['financial', 'statement', 'consolidated']):
        return 'financial_statement'
    else:
        return 'general_table'

### Table Comparison and Analysis Functions

In [32]:
def compare_table_extractions(results: List[TableExtractionResult]) -> List[TableComparison]:
    """Compare table extraction results across different methods"""
    comparisons = []
    
    # Group results by page and table index
    grouped_results = {}
    for result in results:
        key = (result.page_number, result.table_index)
        if key not in grouped_results:
            grouped_results[key] = []
        grouped_results[key].append(result)
    
    # Compare results for each table
    for (page_num, table_idx), method_results in grouped_results.items():
        if len(method_results) < 2:
            continue  # Need at least 2 methods to compare
        
        # Calculate quality scores for each method
        quality_scores = {}
        row_column_preservation = {}
        
        for result in method_results:
            if result.success and not result.raw_data.empty:
                quality_scores[result.method.value] = assess_table_quality(result.raw_data, result.method)
                row_column_preservation[result.method.value] = {
                    'rows': result.rows,
                    'columns': result.columns
                }
        
        # Determine best method
        best_method = None
        best_score = 0
        if quality_scores:
            best_method_name = max(quality_scores.keys(), 
                                 key=lambda k: quality_scores[k]['overall_score'])
            best_score = quality_scores[best_method_name]['overall_score']
            best_method = TableMethod(best_method_name)
        
        # Generate recommendations
        recommendations = generate_recommendations(quality_scores, row_column_preservation)
        
        comparison = TableComparison(
            page_number=page_num,
            table_index=table_idx,
            methods_compared=[result.method for result in method_results],
            best_method=best_method,
            quality_scores=quality_scores,
            row_column_preservation=row_column_preservation,
            recommendations=recommendations
        )
        comparisons.append(comparison)
    
    return comparisons

def generate_recommendations(quality_scores: Dict, row_column_preservation: Dict) -> List[str]:
    """Generate recommendations based on comparison results"""
    recommendations = []
    
    if not quality_scores:
        return ["No successful extractions to compare"]
    
    # Find best and worst methods
    best_method = max(quality_scores.keys(), key=lambda k: quality_scores[k]['overall_score'])
    worst_method = min(quality_scores.keys(), key=lambda k: quality_scores[k]['overall_score'])
    
    recommendations.append(f"Best method: {best_method} (score: {quality_scores[best_method]['overall_score']:.1f})")
    recommendations.append(f"Worst method: {worst_method} (score: {quality_scores[worst_method]['overall_score']:.1f})")
    
    # Method-specific recommendations
    if 'camelot_lattice' in quality_scores and 'camelot_stream' in quality_scores:
        lattice_score = quality_scores['camelot_lattice']['overall_score']
        stream_score = quality_scores['camelot_stream']['overall_score']
        
        if lattice_score > stream_score + 10:
            recommendations.append("Table has clear borders - lattice mode preferred")
        elif stream_score > lattice_score + 10:
            recommendations.append("Table is borderless - stream mode preferred")
        else:
            recommendations.append("Both Camelot modes perform similarly - consider hybrid approach")
    
    # Structure preservation analysis
    if len(row_column_preservation) > 1:
        row_counts = [data['rows'] for data in row_column_preservation.values()]
        col_counts = [data['columns'] for data in row_column_preservation.values()]
        
        if max(row_counts) - min(row_counts) > 2:
            recommendations.append("Significant variation in row count - check for merged cells")
        if max(col_counts) - min(col_counts) > 1:
            recommendations.append("Significant variation in column count - check for column detection")
    
    return recommendations

### Hybrid Extractor Function

In [33]:
def create_hybrid_extractor(pdf_path: str, page_num: int = None) -> List[TableExtractionResult]:
    """Create a hybrid extractor that chooses the best method based on heuristics"""
    results = []
    
    try:
        # First, try to detect if the page has ruling lines
        has_ruling_lines = detect_ruling_lines(pdf_path, page_num)
        
        if has_ruling_lines:
            print(f"  📏 Detected ruling lines - using Camelot lattice mode")
            results.extend(extract_tables_camelot_lattice(pdf_path, page_num))
        else:
            print(f"  📝 No ruling lines detected - using Camelot stream mode")
            results.extend(extract_tables_camelot_stream(pdf_path, page_num))
        
        # Always try pdfplumber as backup
        print(f"  🔧 Running pdfplumber as backup method")
        pdfplumber_results = extract_tables_pdfplumber(pdf_path, page_num)
        results.extend(pdfplumber_results)
        
        # Compare results and select best
        if len(results) > 1:
            comparisons = compare_table_extractions(results)
            if comparisons:
                best_comparison = comparisons[0]  # Assuming first comparison is relevant
                if best_comparison.best_method:
                    print(f"  🏆 Best method identified: {best_comparison.best_method.value}")
        
    except Exception as e:
        print(f"  ❌ Hybrid extraction failed: {e}")
    
    return results

def detect_ruling_lines(pdf_path: str, page_num: int = None) -> bool:
    """Detect if a page has ruling lines (simple heuristic)"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[page_num - 1] if page_num else pdf.pages[0]
            
            # Get lines from the page
            lines = page.lines
            
            # Count horizontal and vertical lines
            horizontal_lines = [line for line in lines if abs(line['y1'] - line['y0']) < 5]
            vertical_lines = [line for line in lines if abs(line['x1'] - line['x0']) < 5]
            
            # Simple heuristic: if we have both horizontal and vertical lines, likely has ruling lines
            has_ruling_lines = len(horizontal_lines) > 5 and len(vertical_lines) > 5
            
            return has_ruling_lines
            
    except Exception as e:
        print(f"  ⚠️ Could not detect ruling lines: {e}")
        return False

### Main Processing Function

In [34]:
def process_pdf_for_tables(pdf_path: str) -> Dict:
    """Process a single PDF for table extraction using all methods"""
    filename = os.path.basename(pdf_path)
    base_name = filename.replace('.pdf', '')
    year = filename.split('_')[2][:4]
    
    print(f"\n�� Processing: {filename}")
    print(f"📅 Year: {year}")
    
    # Create year-specific output directory
    year_output_dir = os.path.join(table_output_dir, year)
    os.makedirs(year_output_dir, exist_ok=True)
    
    all_results = []
    all_comparisons = []
    
    try:
        # Extract tables using all methods
        print("  🔍 Running Camelot lattice extraction...")
        lattice_results = extract_tables_camelot_lattice(pdf_path)
        all_results.extend(lattice_results)
        
        print("  🔍 Running Camelot stream extraction...")
        stream_results = extract_tables_camelot_stream(pdf_path)
        all_results.extend(stream_results)
        
        print("  🔍 Running pdfplumber extraction...")
        pdfplumber_results = extract_tables_pdfplumber(pdf_path)
        all_results.extend(pdfplumber_results)
        
        print("  🔍 Running hybrid extraction...")
        hybrid_results = create_hybrid_extractor(pdf_path)
        
        # Compare results
        print("  📊 Comparing extraction methods...")
        comparisons = compare_table_extractions(all_results)
        all_comparisons.extend(comparisons)
        
        # Save results
        save_table_results(all_results, all_comparisons, year_output_dir, base_name)
        
        # Generate summary
        generate_extraction_summary(all_results, all_comparisons, filename)
        
    except Exception as e:
        print(f"  ❌ Error processing {filename}: {e}")
    
    return {
        'filename': filename,
        'results': all_results,
        'comparisons': all_comparisons,
        'total_tables': len([r for r in all_results if r.success]),
        'successful_methods': len(set(r.method for r in all_results if r.success))
    }

### Save and Analysis Functions

In [35]:
def save_table_results(results: List[TableExtractionResult], comparisons: List[TableComparison], 
                      output_dir: str, base_name: str):
    """Save table extraction results"""
    
    # Save individual CSV files for each successful extraction
    csv_dir = os.path.join(output_dir, f"{base_name}_tables")
    os.makedirs(csv_dir, exist_ok=True)
    
    for result in results:
        if result.success and not result.raw_data.empty:
            csv_filename = f"{base_name}_page{result.page_number}_table{result.table_index}_{result.method.value}.csv"
            csv_path = os.path.join(csv_dir, csv_filename)
            result.raw_data.to_csv(csv_path, index=False)
            print(f"    💾 Saved: {csv_filename}")
    
    # Save comparison results
    comparison_file = os.path.join(output_dir, f"{base_name}_comparison.json")
    comparison_data = {
        'filename': base_name,
        'processing_date': datetime.now().isoformat(),
        'total_extractions': len(results),
        'successful_extractions': len([r for r in results if r.success]),
        'comparisons': [comp.to_dict() for comp in comparisons],
        'method_performance': analyze_method_performance(results)
    }
    
    with open(comparison_file, 'w', encoding='utf-8') as f:
        json.dump(comparison_data, f, indent=2, ensure_ascii=False)
    
    print(f"  Saved comparison analysis: {os.path.basename(comparison_file)}")
    
def analyze_method_performance(results: List[TableExtractionResult]) -> Dict:
    """Analyze performance of different extraction methods"""
    method_stats = {}
    
    for method in TableMethod:
        method_results = [r for r in results if r.method == method]
        if method_results:
            successful = [r for r in method_results if r.success]
            method_stats[method.value] = {
                'total_attempts': len(method_results),
                'successful_extractions': len(successful),
                'success_rate': len(successful) / len(method_results) * 100,
                'avg_extraction_time': np.mean([r.extraction_time for r in successful]) if successful else 0,
                'avg_confidence': np.mean([r.confidence for r in successful if r.confidence]) if successful else None,
                'total_tables_found': len(successful)
            }
    
    return method_stats

def generate_extraction_summary(results: List[TableExtractionResult], comparisons: List[TableComparison], filename: str):
    """Generate and print extraction summary"""
    print(f"\n  📈 EXTRACTION SUMMARY for {filename}")
    print(f"  {'='*60}")
    
    # Method performance
    method_stats = analyze_method_performance(results)
    for method, stats in method_stats.items():
        print(f"  {method.upper()}:")
        print(f"    Success rate: {stats['success_rate']:.1f}% ({stats['successful_extractions']}/{stats['total_attempts']})")
        print(f"    Avg time: {stats['avg_extraction_time']:.2f}s")
        if stats['avg_confidence']:
            print(f"    Avg confidence: {stats['avg_confidence']:.1f}%")
    
    # Best method recommendations
    if comparisons:
        print(f"\n  🏆 RECOMMENDATIONS:")
        for comp in comparisons[:3]:  # Show top 3 comparisons
            if comp.best_method:
                print(f"    Page {comp.page_number}, Table {comp.table_index}: {comp.best_method.value}")
                for rec in comp.recommendations[:2]:  # Show top 2 recommendations
                    print(f"      • {rec}")
    
    print(f"  {'='*60}")

### Main Execution Function

In [36]:
def main():
    """Main function to process all PDFs for table extraction"""
    print("�� Starting Table Extraction Analysis")
    print("="*60)
    
    # Get all PDF files
    pdf_files = glob.glob(os.path.join(raw_pdf_dir, "*.pdf"))
    print(f"📁 Found {len(pdf_files)} PDF files to process")
    
    overall_stats = {
        'total_files': len(pdf_files),
        'processed_files': 0,
        'total_tables_found': 0,
        'method_performance': {},
        'best_methods': []
    }
    
    for pdf_path in pdf_files:
        try:
            result = process_pdf_for_tables(pdf_path)
            overall_stats['processed_files'] += 1
            overall_stats['total_tables_found'] += result['total_tables']
            
        except Exception as e:
            print(f"❌ Failed to process {os.path.basename(pdf_path)}: {e}")
    
    # Print final summary
    print(f"\n�� TABLE EXTRACTION COMPLETED!")
    print(f"📊 Processed {overall_stats['processed_files']}/{overall_stats['total_files']} files")
    print(f"📋 Found {overall_stats['total_tables_found']} total tables")
    print(f"💾 Results saved to: {table_output_dir}")

# Execute the main function
if __name__ == "__main__":
    main()

�� Starting Table Extraction Analysis
📁 Found 3 PDF files to process

�� Processing: MSFT_10-K_20230727_000095017023035122.pdf
📅 Year: 2023
  🔍 Running Camelot lattice extraction...
  🔍 Running Camelot stream extraction...
  🔍 Running pdfplumber extraction...
  🔍 Running hybrid extraction...
  📝 No ruling lines detected - using Camelot stream mode
  🔧 Running pdfplumber as backup method
  📊 Comparing extraction methods...
  Saved comparison analysis: MSFT_10-K_20230727_000095017023035122_comparison.json

  📈 EXTRACTION SUMMARY for MSFT_10-K_20230727_000095017023035122.pdf
  CAMELOT_STREAM:
    Success rate: 0.0% (0/1)
    Avg time: 0.00s
  PDFPLUMBER:
    Success rate: 0.0% (0/1)
    Avg time: 0.00s

  🏆 RECOMMENDATIONS:

�� Processing: MSFT_10-K_20240730_000095017024087843.pdf
📅 Year: 2024
  🔍 Running Camelot lattice extraction...
  🔍 Running Camelot stream extraction...
  🔍 Running pdfplumber extraction...
  🔍 Running hybrid extraction...
  📝 No ruling lines detected - using Camelot 